### Make the Cars Stop in a case of cyclist

In [1]:
import carla
import numpy as np
import cv2
import time
import sys
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev, interp1d
import math

In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]
spawn_points = world.get_map().get_spawn_points()

In [21]:
def carorigin():
    global vehicle, vehicle_start_point
    vehicle_bp = world.get_blueprint_library().filter('*mini*')
    vehicle_start_point = spawn_points[94]
    vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)

In [55]:
def bicycleorigin():
    global bicycle
    bicycle_bp = world.get_blueprint_library().filter('*crossbike*')
    bicycle_start_point = spawn_points[99]

    bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
    
    new_location = bicycle_start_point.location + carla.Location(y=17)
    new_rotation = carla.Rotation(pitch=0, yaw=bicycle_start_point.rotation.yaw + 0, roll=0)

    
    bicyclepos = carla.Transform(new_location, new_rotation)
    bicycle.set_transform(bicyclepos)

In [42]:
def destroy():#Destroying the existing things
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

In [62]:
vehicle = None
bicycle = None
vehicle_start_point = None
destroy()
bicycleorigin()
carorigin()

In [59]:
destroy()

In [20]:
def DrawPointsFor30Sec(world, spawn_points):
    drawn_points = []
    for index, waypoint in enumerate(spawn_points):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        point = world.debug.draw_string(
            waypoint.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=30,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        drawn_points.append(point)

DrawPointsFor30Sec(world, spawn_points)

## Car and Cyclist position
### Car Start: ID: 1
### Car End: ID: 55
### Bycicyle start: ID 108

In [57]:
PREFERRED_SPEED = 30 #km/h
SPEED_THRESHOLD = 2 #When we close to the required speed, it drops the throttle
MAX_STEER_DEGREES = 40

def maintain_speed(s: float) -> float:
    
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8
    else:
        return 0.4

In [61]:
# Make the cyclist go
bicycle.apply_control(carla.VehicleControl(throttle=1))
v = vehicle.get_velocity()
speed = round(3.6*math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
estimated_throttle = maintain_speed(speed)
vehicle.apply_control(carla.VehicleControl(throttle = estimated_throttle))
